In [7]:
import pandas as pd
from medscraper.buscadores import busca_cofepris

In [2]:
url="https://raw.githubusercontent.com/labdatos-se/entregables_meds/main/piloto_seguimiento/levantamiento_21jul2023.csv?token=GHSAT0AAAAAACFMKSSTRJK74QWCDXAZYM54ZF3GURA"

In [3]:
sucia=pd.read_csv(url)
sucia

,id,producto,precio,hora_consulta,farmacia_corto,busqueda,farmacia,pais,product_name,quantity,unit,presentacion,num_prod
0,11740,calquence 100 mg caja con 56 capsulas,176000.00,10:40:15 20/07/2023,farmalisto_mx,acalabrutinib,farmalisto méxico,Mexico,calquence,100.0,mg,capsula,56.0
1,11741,calquence 56 capsulas caja acalabrutinib 100.0 mg,184440.50,10:40:31 20/07/2023,fsanpablo,acalabrutinib,farmacias san pablo,Mexico,calquence,100.0,mg,capsula,56.0
2,11742,calquence 100 mg cap caja con 56,186700.00,10:40:41 20/07/2023,fespecializadas,acalabrutinib,farmacias especializadas,Mexico,calquence,100.0,mg,capsula,56.0
3,11743,calcuencia,150900.13,10:42:31 20/07/2023,pchecker,acalabrutinib,canadianpharmacystore.com,Canada,calcuencia,100.0,mg,capsula,56.0
4,11744,calcuencia,451778.56,10:42:31 20/07/2023,pchecker,acalabrutinib,canadianpharmacystore.com,Canada,calcuencia,100.0,mg,capsula,168.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9144,20884,revostat 10 mg tira de 10 tabletas,19.12,23:53:51 20/07/2023,pharmeasy,vorinostat,pharmeasy,India,revostat,10.0,mg,tableta,10.0
9145,20885,tableta de 20 mg de rozustat,59.13,23:53:51 20/07/2023,pharmeasy,vorinostat,pharmeasy,India,tableta de,20.0,mg,tableta,NaN
9146,20886,novastat 40mg tab 15 s,197.25,23:53:51 20/07/2023,pharmeasy,vorinostat,pharmeasy,India,novastat,40.0,mg,tableta,15.0
9147,20887,hyrostat 10mg tableta 10 s,29.66,23:53:51 20/07/2023,pharmeasy,vorinostat,pharmeasy,India,hyrostat,10.0,mg,tableta,10.0


### Limpieza de datos

In [14]:
#Hacer copia de la base de datos
lienzo=sucia.copy()

In [15]:
def limpieza(df):
    #Primera condición: Eliminar Guadalajara
    df = df[df['farmacia_corto']!='fguadalajara']
    #Segunda condición: busqueda se encuentra en producto
    
    coincidencia = df[df.apply(lambda row: all(word in row['producto'] for word in row['busqueda'].split()), axis=1)]
    #Si id de coincidencia es igual a id de la base de datos, entonces poner 1 en la columna coincidencia
    df['coincidencia'] = df['id'].isin(coincidencia['id']).astype(int)
    return df


In [16]:
lienzo=limpieza(lienzo)
lienzo

,id,producto,precio,hora_consulta,farmacia_corto,busqueda,farmacia,pais,product_name,quantity,unit,presentacion,num_prod,coincidencia
0,11740,calquence 100 mg caja con 56 capsulas,176000.00,10:40:15 20/07/2023,farmalisto_mx,acalabrutinib,farmalisto méxico,Mexico,calquence,100.0,mg,capsula,56.0,0
1,11741,calquence 56 capsulas caja acalabrutinib 100.0 mg,184440.50,10:40:31 20/07/2023,fsanpablo,acalabrutinib,farmacias san pablo,Mexico,calquence,100.0,mg,capsula,56.0,1
2,11742,calquence 100 mg cap caja con 56,186700.00,10:40:41 20/07/2023,fespecializadas,acalabrutinib,farmacias especializadas,Mexico,calquence,100.0,mg,capsula,56.0,0
3,11743,calcuencia,150900.13,10:42:31 20/07/2023,pchecker,acalabrutinib,canadianpharmacystore.com,Canada,calcuencia,100.0,mg,capsula,56.0,0
4,11744,calcuencia,451778.56,10:42:31 20/07/2023,pchecker,acalabrutinib,canadianpharmacystore.com,Canada,calcuencia,100.0,mg,capsula,168.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9144,20884,revostat 10 mg tira de 10 tabletas,19.12,23:53:51 20/07/2023,pharmeasy,vorinostat,pharmeasy,India,revostat,10.0,mg,tableta,10.0,0
9145,20885,tableta de 20 mg de rozustat,59.13,23:53:51 20/07/2023,pharmeasy,vorinostat,pharmeasy,India,tableta de,20.0,mg,tableta,NaN,0
9146,20886,novastat 40mg tab 15 s,197.25,23:53:51 20/07/2023,pharmeasy,vorinostat,pharmeasy,India,novastat,40.0,mg,tableta,15.0,0
9147,20887,hyrostat 10mg tableta 10 s,29.66,23:53:51 20/07/2023,pharmeasy,vorinostat,pharmeasy,India,hyrostat,10.0,mg,tableta,10.0,0


In [17]:
#Contar ceros y unos de la columna coincidencia
lienzo['coincidencia'].value_counts()

0    7585
1    1564
Name: coincidencia, dtype: int64